# Prototype application for NIRSpec MOS preimaging planner

This application is intended to demonstrate some minimal functionality for the preimaging planning tool, implemented as an interactive Jupyter notebook with IPyWidget buttons and an Imviz application for visualization.  The FITS data, FOV regions, and source catalog are pre-loaded in this prototype; they will be dynamically identified, calculated, and loaded in the real application.

Try pressing the show/hide buttons below to demonstrate the custom display controls. Use Imviz tools directly for zoom, scale, colormap, etc.

In [ ]:
import os
import warnings

from astropy.coordinates import SkyCoord
from astropy.table import Table
from IPython.display import display, HTML
import ipywidgets as ipw
from jdaviz import Imviz
import numpy as np
from regions import Regions

In [ ]:
# set up viewer sizing for voila app
display(HTML("<style>.jdaviz__content--not-in-notebook {min-height: 80vh; max-height:80vh}</style>"))

# set working directory if called from voila app
os.chdir(os.environ.get('JDAVIZ_START_DIR', '.'))

In [ ]:
# start imviz viewer
viz = Imviz()
viewer = viz.default_viewer

In [ ]:
# load FITS data
image_name = 'hlsp_legus_hst_acs_ngc5194-ngc5195-mosaic_f814w_v1_sci.fits'
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    viz.load_data(image_name)

In [ ]:
# load the NIRSpec FOV
reg_name = 'ds9-msa.reg'
raw_regs = Regions.read(reg_name, format='ds9')

viz.load_regions(raw_regs)
i = 0
for subset in viz.app.data_collection.subset_groups:
    subset.style.color = 'red'
    subset.style.alpha = 0
    subset.label = f'NIRSpec {i}'
    i += 1

In [ ]:
# load the NIRCam FOVs
reg_name = 'ds9-long-no.reg'
raw_regs = Regions.read(reg_name, format='ds9')

viz.load_regions(raw_regs)
i = 0
for subset in viz.app.data_collection.subset_groups:
    if subset.label.startswith('Masked'):
        subset.style.color = 'blue'
        subset.style.alpha = 0
        subset.label = f'NIRCam Long {i}'
        i += 1
        
reg_name = 'ds9-short-no.reg'
raw_regs = Regions.read(reg_name, format='ds9')

viz.load_regions(raw_regs)
i = 0
for subset in viz.app.data_collection.subset_groups:
    if subset.label.startswith('Masked'):
        subset.style.color = 'green'
        subset.style.alpha = 0
        subset.label = f'NIRCam Short {i}'
        i += 1

In [ ]:
# load the source catalog
cat_name = 'm51.radec'
catalog = Table.read(cat_name, format='ascii')

filler = (catalog.columns[2] == 'F')
primary = ~filler

primary_coord = Table({'coord': [SkyCoord(ra=catalog.columns[0][primary],
                                          dec=catalog.columns[1][primary],
                                          unit="deg")]})
filler_coord = Table({'coord': [SkyCoord(ra=catalog.columns[0][filler],
                                         dec=catalog.columns[1][filler],
                                         unit="deg")]})
viewer.marker = {'color': 'red', 'alpha': 0.5, 'markersize': 10, 'fill': False}
viewer.add_markers(primary_coord, use_skycoord=True, marker_name='primary_markers')

viewer.marker = {'color': 'yellow', 'alpha': 0.3, 'markersize': 10, 'fill': False}
viewer.add_markers(filler_coord, use_skycoord=True, marker_name='filler_markers')

for lyr in viewer.layers:
    if 'markers' in lyr.layer.label:
        lyr.visible = False

In [ ]:
# define FOV show/hide buttons
class FOVButton(ipw.Button):
    def __init__(self, *args, **kwargs):
        self.value = kwargs.pop('value', '')
        super().__init__(*args, **kwargs)

def on_fov_clicked(b):
    if b.description.startswith('Show'):
        b.description = f'Hide {b.value} FOV'
        for subset in viz.app.data_collection.subset_groups:
            if b.value in subset.label:
                subset.style.alpha = 0.5
    else:            
        b.description = f'Show {b.value} FOV'
        for subset in viz.app.data_collection.subset_groups:
            if b.value in subset.label:
                subset.style.alpha = 0
                
fov_buttons = []
for name in ['NIRSpec', 'NIRCam Short', 'NIRCam Long']:
    button = FOVButton(description=f'Show {name} FOV', value=name, layout=ipw.Layout(width='auto'))
    button.on_click(on_fov_clicked)
    fov_buttons.append(button)

In [ ]:
# define catalog show/hide button
def on_cat_clicked(b):
    if b.description.startswith('Show'):
        b.description = 'Hide M51 Catalog'
        for lyr in viewer.layers:
            if 'markers' in lyr.layer.label:
                lyr.visible = True

    else:
        b.description = 'Show M51 Catalog'
        for lyr in viewer.layers:
            if 'markers' in lyr.layer.label:
                lyr.visible = False
                
cat_button = ipw.Button(description='Show M51 Catalog', layout=ipw.Layout(width='auto'))
cat_button.on_click(on_cat_clicked)

In [ ]:
# set layout
button_layout = ipw.Layout(display='flex', flex_flow='row', justify_content='flex-start')
box_layout = ipw.Layout(display='flex', flex_flow='column', align_items='stretch', width='90vw', height='90vh')

buttons = ipw.Box(children=fov_buttons + [cat_button], layout=button_layout)
box = ipw.Box(children=[buttons, viz.app], layout=box_layout)

In [ ]:
# display widgets
display(box)